# Packages

In [1]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

import rasterio

# Load Data
We'll focus on the 2010s for now. Maybe change to longer time period after this.

In [46]:
compustat = pd.read_csv("data/compustatChanges_2010s.csv").drop(columns = ['Unnamed: 0'])


In [47]:
compustat = compustat[~compustat.qtr.isna()]

In [48]:
compustat.qtr

0         1.0
1         2.0
2         3.0
3         4.0
4         1.0
         ... 
382293    4.0
382294    1.0
382295    2.0
382296    3.0
382297    4.0
Name: qtr, Length: 382291, dtype: float64

In [49]:
compustat['fquarter'] = "q" + compustat.qtr.astype(int).astype(str) + "y" + compustat.year.astype(int).astype(str)
compustat.rename(columns = {'tic': 'TICKER'}, inplace = True)

In [50]:
compustat

,year,qtr,gvkey,companyName,TICKER,naics,curcdq,incomeChange,revenueChange,costChange,inventoryChange,fquarter
0,2010,1.0,1004,AAR CORP,AIR,423860.0,USD,0.213983,0.200565,0.178258,0.055631,q1y2010
1,2010,2.0,1004,AAR CORP,AIR,423860.0,USD,0.045617,0.059577,0.017636,0.086776,q2y2010
2,2010,3.0,1004,AAR CORP,AIR,423860.0,USD,0.153198,0.166276,0.129456,0.169270,q3y2010
3,2010,4.0,1004,AAR CORP,AIR,423860.0,USD,-0.398739,0.174283,0.136013,0.182304,q4y2010
4,2011,1.0,1004,AAR CORP,AIR,423860.0,USD,0.096386,0.133883,0.101523,0.104150,q1y2011
...,...,...,...,...,...,...,...,...,...,...,...,...
382293,2018,4.0,345980,CONTEXTLOGIC INC,WISH,454110.0,USD,-0.403846,-0.666667,-0.314815,NaN,q4y2018
382294,2018,1.0,347085,KAROOOOO LTD,KARO,518210.0,USD,NaN,NaN,NaN,NaN,q1y2018
382295,2018,2.0,347085,KAROOOOO LTD,KARO,518210.0,USD,NaN,NaN,NaN,NaN,q2y2018
382296,2018,3.0,347085,KAROOOOO LTD,KARO,518210.0,USD,NaN,NaN,NaN,NaN,q3y2018


In [51]:
hq = pd.read_csv("data/allData_byQuarter.csv").drop(columns = ['Unnamed: 0'])
hq

,company,TICKER,fquarter,precipQuartOverall,precipQuartState,precipQuartStateMonth,tmaxQuartOverall,tmaxQuartState,tmaxQuartStateMonth
0,5G ACQUISITIONS LLC,SPDC,q1y2019,6.0,5.0,8.0,1.0,0.0,1.0
1,5G ACQUISITIONS LLC,SPDC,q2y2019,9.0,9.0,6.0,0.0,0.0,0.0
2,5G ACQUISITIONS LLC,SPDC,q3y2019,7.0,6.0,5.0,0.0,0.0,0.0
3,5G ACQUISITIONS LLC,SPDC,q4y2019,5.0,5.0,5.0,5.0,0.0,5.0
4,8X8 INC,EGHT,q1y2010,6.0,14.0,7.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
49695,ZYNGA INC,ZNGA,q3y2017,0.0,0.0,1.0,2.0,2.0,2.0
49696,ZYNGA INC,ZNGA,q4y2013,0.0,3.0,1.0,0.0,0.0,0.0
49697,ZYNGA INC,ZNGA,q4y2014,9.0,15.0,12.0,0.0,1.0,0.0
49698,ZYNGA INC,ZNGA,q4y2016,5.0,16.0,14.0,0.0,0.0,0.0


In [52]:
compustatWithHQ = compustat.merge(hq)

In [54]:
compustatWithHQ.head()

,year,qtr,gvkey,companyName,TICKER,naics,curcdq,incomeChange,revenueChange,costChange,inventoryChange,fquarter,company,precipQuartOverall,precipQuartState,precipQuartStateMonth,tmaxQuartOverall,tmaxQuartState,tmaxQuartStateMonth
0,2010,1.0,1050,CECO ENVIRONMENTAL CORP,CECE,333413.0,USD,11.969072,0.026698,0.018600,-0.098933,q1y2010,CECO ENVIRONMENTAL CORP,4.0,4.0,5.0,0.0,0.0,0.0
1,2010,2.0,1050,CECO ENVIRONMENTAL CORP,CECE,333413.0,USD,2.355670,-0.064383,-0.094254,0.054267,q2y2010,CECO ENVIRONMENTAL CORP,5.0,5.0,4.0,11.0,7.0,11.0
2,2010,3.0,1050,CECO ENVIRONMENTAL CORP,CECE,333413.0,USD,2.395315,-0.026964,-0.107930,0.072734,q3y2010,CECO ENVIRONMENTAL CORP,1.0,1.0,1.0,20.0,41.0,20.0
3,2010,4.0,1050,CECO ENVIRONMENTAL CORP,CECE,333413.0,USD,2.690444,0.021843,-0.066956,0.172967,q4y2010,CECO ENVIRONMENTAL CORP,7.0,6.0,6.0,13.0,0.0,13.0
4,2011,1.0,1050,CECO ENVIRONMENTAL CORP,CECE,333413.0,USD,0.625596,-0.081488,-0.168902,-0.213420,q1y2011,CECO ENVIRONMENTAL CORP,5.0,5.0,7.0,9.0,0.0,9.0


In [53]:
compustatWithHQ.to_csv("data/compustatWithHQ.csv")